<a href="https://colab.research.google.com/github/DanilsonCorreia/Mini-Project-Sistemas-Inteligentes/blob/main/Mini_Project_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths for dataset
train_data_path = "/path/to/training/dataset"
eval_data_path = "/path/to/evaluation/dataset"
result_file = "result.txt"

# Data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

eval_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load datasets
train_dataset = datasets.ImageFolder(train_data_path, transform=train_transforms)
eval_dataset = datasets.ImageFolder(eval_data_path, transform=eval_transforms)

# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# Load pre-trained model (MobileNetV2) and modify for binary classification
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 1)  # Binary classification
model = model.to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_f1 = 0.0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        y_true, y_pred = [], []

        # Training step
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Collect predictions for F1 score
            preds = torch.sigmoid(outputs).cpu().detach().numpy() > 0.5
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds)

        # Calculate F1 score for training
        train_f1 = f1_score(y_true, np.array(y_pred))

        # Validation step
        model.eval()
        val_loss = 0.0
        val_y_true, val_y_pred = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Collect predictions for F1 score
                preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
                val_y_true.extend(labels.cpu().numpy())
                val_y_pred.extend(preds)

        # Calculate F1 score for validation
        val_f1 = f1_score(val_y_true, np.array(val_y_pred))

        print(f"Epoch {epoch + 1} | Train Loss: {train_loss / len(train_loader):.4f}, Train F1: {train_f1:.4f}, "
              f"Val Loss: {val_loss / len(val_loader):.4f}, Val F1: {val_f1:.4f}")

        # Save the best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Best model saved with F1: {best_f1:.4f}")

    print("Training complete!")
    return model

# Train the model
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Load the best model for evaluation
model.load_state_dict(torch.load("best_model.pth"))

# Generate predictions on the evaluation set
model.eval()
results = []
with torch.no_grad():
    for images, labels in eval_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
        for img, pred in zip(labels, preds):
            results.append(f"{img} {int(pred[0])}")

# Save predictions to result.txt
with open(result_file, "w") as f:
    f.write("\n".join(results))

print(f"Results saved to {result_file}")

Using device: cpu


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/training/dataset'